# Rule Mining Notebook

This is a sample notebook for our procedure to extract decision rules from tree based models including decision trees, random forests, and fair decision trees. Note this notebook requires boths scikit-learn and fairlearn, and is based around binerized data.

In [ ]:
import pandas as pd
import numpy as np
import time
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from fairlearn.reductions import ExponentiatedGradient, EqualizedOdds, TruePositiveRateParity, ErrorRateParity

## Helper Functions

In [ ]:
#Helper functions for computing fairness/accuracy
def compute_TPR_GAP(preds, Y, group):
    res = {}
    res['TPR'] = sum(preds[Y])/len(preds[Y])
    res['TPR_1'] = sum(preds[Y & group])/len(preds[Y & group])
    res['TPR_2'] = sum(preds[Y & ~group])/len(preds[Y & ~group])
    res['TPR_GAP'] = abs(res['TPR_1'] - res['TPR_2'])
    return res

def compute_TNR_GAP(preds, Y, group):
    res = {}
    res['TNR'] = sum(~preds[~Y])/len(preds[~Y])
    res['TNR_1'] = sum(~preds[~Y & group])/len(preds[~Y & group])
    res['TNR_2'] = sum(~preds[~Y & ~group])/len(preds[~Y & ~group])
    res['TNR_GAP'] = abs(res['TNR_1'] - res['TNR_2'])
    return res

def compute_ACC_GAP(preds, Y, group):
    res = {}
    res['ACC'] = sum(preds == Y)/len(Y)
    res['ACC_1'] = sum(preds[group] == Y[group])/len(Y[group])
    res['ACC_2'] = sum(preds[~group] == Y[~group])/len(Y[~group])
    res['ACC_GAP'] = abs(res['ACC_1'] - res['ACC_2'])
    return res
    
def compute_EqOpp(preds, Y, group):
    return compute_TPR_GAP(preds, Y, group)

def compute_EqOd(preds, Y, group):
    return compute_TPR_GAP(preds, Y, group).update(compute_TNR_GAP(preds, Y, group) )

def compute_AccDisp(preds, Y, group):
    return compute_ACC_GAP(preds, Y, group)

def compute_fairness(preds,Y,group):
    res = compute_TPR_GAP(preds, Y, group)
    res.update(compute_TNR_GAP(preds, Y, group))
    res.update(compute_ACC_GAP(preds,Y,group))
    return res


In [ ]:
#Helper function rule to extract decision rules from a tree
def extract_rules(tree, X, orig_features, subset, subset_c):
    #Extract Relevant features about the tree
    leaf_id = tree.apply(X)
    node_indicator = tree.decision_path(X)
    threshold = tree.tree_.threshold
    feature = tree.tree_.feature
    
    #Get one sample for each leaf node
    samples = pd.DataFrame(leaf_id).drop_duplicates().index.to_numpy()
    
    #Set up output rule list
    rules = []
    
    #Loop over leaf node samples
    for sample_id in samples:
        #Create rule
        rule = np.zeros(len(orig_features) - 1)
        
        #Pulls all nodes that the sample touches
        node_index = node_indicator.indices[node_indicator.indptr[sample_id]:
                                        node_indicator.indptr[sample_id + 1]]
        
        for node_id in node_index:
            # continue to the next node if it is a leaf node
            if leaf_id[sample_id] == node_id:
                #Pull prediction for that node (majority class)
                leaf_val = np.argmax(tree.tree_.value[node_id])
                continue
                
            # check if value of the split feature for sample is below threshold
            if (X[sample_id, feature[node_id]] <= threshold[node_id]):
                #Get the complement of the feature
                feat_index = orig_features.index(subset_c[feature[node_id]])
            else:
                #If it's 'true' get the name of the feature
                feat_index = orig_features.index(subset[feature[node_id]])
            
            #Set the associated feature in thee rule ot be 1
            rule[feat_index] = 1
        
        #If it's a rule for the positive class, add it
        if leaf_val == 1:
            rules.append(rule)
            
    return np.array(rules)


# Decision Tree

Code to train and extract rules from a simple decision tree.

In [ ]:
for i in range(10):
    print('******** FOLD: %d ********'%i)
    for data_set in ['adult','compas','default']:
        print('******** Dataset: %s********'%data_set)
        train  = pd.read_csv('split_data/bin_'+data_set+'_train_%d.csv'%i)
        orig_features = list(train.columns)
        subset = [x for x in orig_features if ('.1' not in x) and ('*' not in x)]
        subset_c = [x for x in orig_features if x not in subset]

        train_sub = train[subset]
        firstFlag = True
        for depth in [2, 5, 10, 15, 20]:
            classifier = DecisionTreeClassifier(max_depth=depth)
            classifier.fit(train_sub.drop('Y',axis=1), train['Y'])
            extracted_rules = extract_rules(classifier, train_sub.drop('Y',axis=1).to_numpy(), 
                                        orig_features, subset, subset_c)
            if len(extracted_rules) == 0:
                continue
                
            if firstFlag:
                rules = extracted_rules
                firstFlag = False
            else:
                rules = np.concatenate([rules, extracted_rules])

        np.save('rule_mining/dt_'+data_set+'_fold_%d'%i, rules)       

# Random Forest
Code to train and extract rules from a random forset.

In [ ]:
for i in range(10):
    print('******** FOLD: %d ********'%i)
    for data_set in ['adult','compas','default']:
        print('******** Dataset: %s********'%data_set)
        train  = pd.read_csv('split_data/bin_'+data_set+'_train_%d.csv'%i)
        orig_features = list(train.columns)
        subset = [x for x in orig_features if ('.1' not in x) and ('*' not in x)]
        subset_c = [x for x in orig_features if x not in subset]

        train_sub = train[subset]
        firstFlag = True
        for depth in [2, 5, 10, 15, 20]:
            classifier = RandomForestClassifier(n_estimators = 10, max_depth=depth)
            classifier.fit(train_sub.drop('Y',axis=1), train['Y'])
            for tree in classifier.estimators_:
                extracted_rules = extract_rules(tree, train_sub.drop('Y',axis=1).to_numpy(), 
                                        orig_features, subset, subset_c)
                if len(extracted_rules) == 0:
                    continue
                    
                if firstFlag:
                    rules = extracted_rules
                    firstFlag = False
                else:
                    rules = np.concatenate([rules, extracted_rules])

        np.save('rule_mining/rf_'+data_set+'_fold_%d'%i, rules)       

# Fair DT
This code chunk trains a fair decision tree classifier using the fairlearn package (specifically exponentiated gradient)

In [ ]:
#Equalized Odds
results = []
datasets = ['default','adult','compas']
protected_features = {'compas': 'race', 
                      'adult': 'gender',
                      'default': 'X2'
                     }

epsilon_ranges = {'compas': [0, 0.05, 0.1, 0.2, 0.5],
                      'adult': [0, 0.02, 0.05, 0.1, 0.5],
                      'default': [0, 0.01, 0.02 , 0.05, 0.5]
                     }
fairness_metrics = ['EqualizedOdds', 'EqOp']

for i in range(10):
    print('***** FOLD %d ******'%i)
    for dataset in datasets:
        print('**** DATASET %s ******'%dataset)
        group_var = protected_features[dataset]
        
        train  = pd.read_csv('split_data/bin_'+dataset+'_train_%d.csv'%i).assign(train = True)
        test = pd.read_csv('split_data/bin_'+dataset+'_test_%d.csv'%i).assign(train = False)
        orig_features = list(train.columns)
        subset = [x for x in orig_features if ('.1' not in x) and ('*' not in x)]
        subset_c = [x for x in orig_features if x not in subset]
        firstFlag = True
        
        train_sub = train[subset]
        test_sub = test[subset]

        for fairMet in fairness_metrics:
            for eps in epsilon_ranges[dataset]:
                for hp in np.linspace(1, 21, 5):
                    print('Epsilon: %f'%eps)
                    
                    if fairMet == 'EqualizedOdds':
                        constraint = EqualizedOdds(difference_bound = eps)
                    elif fairMet == 'EqOp':
                        constraint = TruePositiveRateParity(difference_bound = eps)
                    elif fairMet == 'AccDisp':
                        constraint = ErrorRateParity(difference_bound = eps)
                    
                    classifier = DecisionTreeClassifier(max_depth=hp)
                    mitigator = ExponentiatedGradient(classifier, constraint)

                    start_time = time.time()
                    mitigator.fit(train_sub.drop('Y',axis=1), train_sub['Y'], sensitive_features=train[group_var])
                    end_time = time.time() - start_time
                    
                    for tree in mitigator.predictors_:
                        extracted_rules = extract_rules(tree, train_sub.drop('Y',axis=1).to_numpy(), 
                                                        orig_features, subset, subset_c)
                        if len(extracted_rules) == 0:
                            print('no rules')
                        elif firstFlag:
                            rules = extracted_rules
                            firstFlag = False
                        else:
                            rules = np.concatenate([rules, extracted_rules])

                    preds = mitigator.predict(test_sub.drop('Y',axis=1)).astype(np.bool)
                    y = test['Y'].to_numpy()
                    group = test[group_var].to_numpy()

                    res = compute_fairness(preds, y, group)
                    res['epsilon'] = eps
                    res['fold'] = i
                    res['fairnessCriteria'] = fairMet
                    res['algo'] = 'exp_gradient_decision_tree'
                    res['hp'] = hp
                    res['data_set'] = dataset
                    res['train_time'] = end_time
                    results.append(res)
        np.save('rule_mining/fairdt_'+data_set+'_fold_%d'%i, rules)       